<a href="https://colab.research.google.com/github/Raresney/cv/blob/main/Proiect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Pasul 2

In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 7.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == 

In [2]:
#curatare set de date HuggingFace
import re
from datasets import load_dataset, Dataset, Features, Value
import time

# --- functii de curatare  ---

def curata_problem_statement(text):
    """Curatarea textului unei probleme (care vine din coloana 'prompt')."""
    if not isinstance(text, str):
        return ""
    text = text.lower()
    text = text.replace('$', '')
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    if text.endswith(('.', '?')):
        text = text[:-1].strip()
    return text

def curata_python_solution(text):
    """Curatarea solutiei Python (care vine din coloana 'completion')."""
    if not isinstance(text, str):
        return ""
    text = text.strip()
    return text

# --- functie pentru dedublicare ---

def get_unique_indices(data_list):
    """returneaza indicii primelor aparitii ale elementelor unice dintr-o lista"""
    seen = set()
    indices = []
    for idx, item in enumerate(data_list):
        if item not in seen:
            seen.add(item)
            indices.append(idx)
    return indices

# --- procesarea setului de date ---

# 1. incarcam setul de date fara a specifica 'features', permitand bibliotecii sa detecteze automat coloanele corecte.
print("1. Incarcare set de date")
start_time = time.time()
dataset = None
try:
    # incercam sa incarcam split-ul 'train', lasand 'features' să fie inferat
    dataset = load_dataset("sdiazlor/math-python-reasoning-dataset", split="train")
    print("   Încărcat cu succes split='train'.")
except Exception as e:
    print(f"   Nu am putut încărca split='train', încerc fără split specificat (probabil default='train'). Eroare: {e}")
    try:
        # daca 'train' esueaza, incercam fara a specifica split-ul (de obicei ia 'train' ca default)
        dataset = load_dataset("sdiazlor/math-python-reasoning-dataset")
        # daca datasetul are mai multe splituri, va încarca un DatasetDict ==> selectam 'train'.
        if isinstance(dataset, dict) and "train" in dataset:
             print("   Dataset încărcat ca dict, selectez split='train'.")
             dataset = dataset['train']
        elif isinstance(dataset, dict):
             print(f"   Dataset încărcat ca dict, dar nu conține split='train'. Splituri disponibile: {list(dataset.keys())}")
             dataset = None
        else:
             print("Dataset încărcat cu succes (probabil default='train').")

    except Exception as e2:
        print(f"   Nu am putut încărca setul de date. Eroare finală: {e2}")
        # exit() # a se decomenta daca vrei sa opresti scriptul complet

if dataset is None:
    print("\n!!! EROARE FATALĂ: Nu s-a putut încărca setul de date. Verifică conexiunea și numele setului de date.")
else:
    load_time = time.time() - start_time
    print(f"   Set de date incărcat în {load_time:.2f} secunde.")
    print(f"   Numar initial de exemple: {len(dataset)}")
    print(f"   Coloane detectate: {dataset.column_names}")
    print(f"   Exemplu date originale (folosind coloanele reale):\n{dataset[0]}\n")

    # 2. Aplicam functiile de curatare folosind .map()
    # Adaptam maparea pentru a citi din 'prompt' și 'completion'
    print("2. Curatare date (prompt -> problem_cleaned, completion -> solution_cleaned)...")
    start_time = time.time()
    dataset_curatat = dataset.map(
        lambda exemplu: {
            'problem_cleaned': curata_problem_statement(exemplu['prompt']),
            'solution_cleaned': curata_python_solution(exemplu['completion'])
        },
        num_proc=4, # pot fi ajustate
        remove_columns=dataset.column_names
    )
    map_time = time.time() - start_time
    print(f"   Curatare aplicata in {map_time:.2f} secunde.")
    print(f"   Coloane după curatare: {dataset_curatat.column_names}")
    print(f"   Exemplu date după curatare:\n{dataset_curatat[0]}\n")

    # 3. Identificam și eliminam duplicatele bazate pe 'problem_cleaned'
    print("3. Identificare si eliminare duplicate bazate pe 'problem_cleaned'...")
    start_time = time.time()

    # extragem coloana pe care ne bazam pentru deduplicare
    probleme_curatate_lista = dataset_curatat['problem_cleaned']

    # gasim indicii unici
    indici_unici = get_unique_indices(probleme_curatate_lista)

    num_duplicates = len(dataset_curatat) - len(indici_unici)
    dedup_time = time.time() - start_time
    print(f"   Identificare indici unici în {dedup_time:.2f} secunde.")
    print(f"   Număr de duplicate eliminate: {num_duplicates}")

    # 4. Selectam doar randurile unice pentru a crea setul de date final
    print("4. Creare set de date final deduplicat...")
    start_time = time.time()
    dataset_final = dataset_curatat.select(indici_unici)
    select_time = time.time() - start_time
    print(f"   Selectare rânduri unice în {select_time:.2f} secunde.")
    print(f"   Număr final de exemple: {len(dataset_final)}")

    print("\n--- Procesare completă ---")
    print(f"Numar inițial de probleme: {len(dataset)}") # Folosim len(dataset) original
    print(f"Numar final de probleme unice (după curatare și deduplicare): {len(dataset_final)}")



    # afisam un exemplu final
    print("\nExemplu din setul de date final:")
    if len(dataset_final) > 4:
        print(dataset_final[4])
    elif len(dataset_final) > 0:
        print(dataset_final[0])
    else:
        print("Dataset final este gol.")


    # --- salvare ca CSV ---
    output_csv_filename = "math_python_dataset_curatat.csv"
    print(f"\nSalvare set de date curățat ca fișier CSV: '{output_csv_filename}'...")
    try:
        # index=False este important pentru a nu scrie indexul intern al dataset-ului ca o coloană în CSV
        dataset_final.to_csv(output_csv_filename, index=False, encoding='utf-8')
        print(f"   Set de date salvat cu succes ca '{output_csv_filename}'.")
        # in Colab, fisierul va aparea in panoul "Files" din stanga dupa un timp scurt.
    except Exception as e:
        print(f"   EROARE la salvarea fișierului CSV: {e}")
    # print("\nSalvare set de date curățat...")
    # dataset_final.save_to_disk("./math_python_dataset_curatat")
    # print("   Set de date salvat în directorul './math_python_dataset_curatat'")

1. Incarcare set de date


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.23k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.12M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/500 [00:00<?, ? examples/s]

   Încărcat cu succes split='train'.
   Set de date incărcat în 3.95 secunde.
   Numar initial de exemple: 500
   Coloane detectate: ['prompt', 'completion', 'system_prompt']
   Exemplu date originale (folosind coloanele reale):
{'prompt': 'Define a function that checks whether a word is a palindrome or not. \n\n', 'completion': '<think>\nOkay, so I need to write a Python function called is_palindrome that checks if a word is a palindrome. Hmm, let me think about how to approach this.\n\nFirst, what\'s a palindrome? A palindrome reads the same forwards and backwards. So for a word like "madam," it should return True because reversing it gives the same string.\n\nI remember that strings in Python are immutable, meaning I can\'t change them once they\'re created. That might be helpful if I decide to reverse the string later. But maybe I don\'t need to do that right now. Let\'s see.\n\nTo check if a word is a palindrome, one approach is to compare the original word with its reverse. So, I

Map (num_proc=4):   0%|          | 0/500 [00:00<?, ? examples/s]

   Curatare aplicata in 0.66 secunde.
   Coloane după curatare: ['problem_cleaned', 'solution_cleaned']
   Exemplu date după curatare:
{'problem_cleaned': 'define a function that checks whether a word is a palindrome or not', 'solution_cleaned': '<think>\nOkay, so I need to write a Python function called is_palindrome that checks if a word is a palindrome. Hmm, let me think about how to approach this.\n\nFirst, what\'s a palindrome? A palindrome reads the same forwards and backwards. So for a word like "madam," it should return True because reversing it gives the same string.\n\nI remember that strings in Python are immutable, meaning I can\'t change them once they\'re created. That might be helpful if I decide to reverse the string later. But maybe I don\'t need to do that right now. Let\'s see.\n\nTo check if a word is a palindrome, one approach is to compare the original word with its reverse. So, I can take the input word, convert it into a list of characters, and then join them ba

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

   Set de date salvat cu succes ca 'math_python_dataset_curatat.csv'.


In [ ]:
#Pasul 3 alegerea modelului

In [3]:
!pip install datasets sentence-transformers faiss-cpu torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 62.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 97.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 64.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [4]:
import pandas as pd

csv_filename = "math_python_dataset_curatat.csv"
try:
    df = pd.read_csv(csv_filename)
    #extragem doar coloana cu problemele curatate intr-o lista
    lista_probleme_curatate = df['problem_cleaned'].tolist()
    print(f"Am încărcat {len(lista_probleme_curatate)} probleme din coloana 'problem_cleaned'.")
    # print("Primele 5 probleme:", lista_probleme_curatate[:5])
except FileNotFoundError:
    print(f"EROARE: Fișierul '{csv_filename}' nu a fost găsit.")
    exit()
except KeyError:
    print(f"EROARE: Coloana 'problem_cleaned' nu a fost găsită în '{csv_filename}'.")
    exit()
except Exception as e:
    print(f"EROARE la citirea CSV-ului: {e}")
    exit()

Am încărcat 491 probleme din coloana 'problem_cleaned'.


In [5]:
modele_de_testat = ['all-MiniLM-L6-v2', 'BAAI/bge-base-en-v1.5']

In [6]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import time
import ast # Necesar doar dacă încarci referința din CSV (Varianta B)

# --- Configurări ---
dataset_csv_filename = "math_python_dataset_curatat.csv"
modele_de_testat = ['all-MiniLM-L6-v2', 'BAAI/bge-base-en-v1.5']
K_SEARCH = 5 #vecinii cei mai apropiati sa returneze căutarea

# --- Pasul 1: Încarcă datele principale (doar problemele curățate) ---
print(f"1. Încărcare probleme din '{dataset_csv_filename}'...")
try:
    df = pd.read_csv(dataset_csv_filename)
    lista_probleme_curatate = df['problem_cleaned'].fillna("").astype(str).tolist() # Asigură stringuri, tratează NaN
    if not lista_probleme_curatate:
         raise ValueError("Lista de probleme este goală după încărcare.")
    print(f"   Am încărcat {len(lista_probleme_curatate)} probleme.")
except FileNotFoundError:
    print(f"EROARE: Fișierul '{dataset_csv_filename}' nu a fost găsit.")
    exit()
except KeyError:
    print(f"EROARE: Coloana 'problem_cleaned' nu a fost găsită în '{dataset_csv_filename}'.")
    exit()
except Exception as e:
    print(f"EROARE la citirea CSV-ului principal: {e}")
    exit()

# --- Pasul 2: Definește sau încarcă setul de referință ---
print("\n2. Definire/Încărcare set de referință...")

# ALEGE UNA DIN VARIANTELE A SAU B DE MAI SUS ȘI PUNE CODUL AICI
# Exemplu folosind Varianta A (hardcodat):
# ===========================================================
# PASUL 3 - DEFINIRE SET REFERINȚĂ (Exemplu - Modifică Aici!)
# ===========================================================
set_referinta = [
    { "query_index": 10, "query_text": "what is the sum of 5 and 7", "expected_indices": [25, 150] },
    { "query_index": 45, "query_text": "multiply 6 by 3", "expected_indices": [90] },
    { "query_index": 82, "query_text": "if x = 10 and y = 4 what is x - y", "expected_indices": [120, 30] },
    { "query_index": 15, "query_text": "calculate area of rectangle length 8 width 2", "expected_indices": [200] },
    # Adaugă aici restul intrărilor tale... Asigură-te că indicii sunt corecți!
]
if not set_referinta:
     print("AVERTISMENT: Setul de referință este gol. Evaluarea nu va fi posibilă.")
else:
     print(f"   Set de referință definit/încărcat cu {len(set_referinta)} intrări.")
# ===========================================================


# --- Pasul 3 & 4: Iterează, Testează și Evaluează Modelele ---
print(f"\n3. Testare modele de embedding (k={K_SEARCH})...")

results = {} # Dicționar pentru a stoca rezultatele fiecărui model

for model_name in modele_de_testat:
    print(f"\n--- Testare Model: {model_name} ---")
    results[model_name] = {
        "encoding_time": None,
        "avg_search_time": None,
        "hits_at_k": 0,
        "total_queries": len(set_referinta) if set_referinta else 0,
        "precision_at_k_sum": 0.0,
        "recall_at_k_sum": 0.0
    }

    try:
        # 3a. Încarcă modelul
        print("   Încărcare model...")
        start_load = time.time()
        model = SentenceTransformer(model_name)
        print(f"   Model încărcat în {time.time() - start_load:.2f} sec.")

        # 3b. Generează embeddings pentru tot setul de date
        print(f"   Generare embeddings pentru {len(lista_probleme_curatate)} probleme...")
        start_encode = time.time()
        embeddings = model.encode(lista_probleme_curatate, show_progress_bar=True, normalize_embeddings=True) # Normalizarea e bună pentru distanța cosinus/produs scalar
        encoding_time = time.time() - start_encode
        results[model_name]["encoding_time"] = encoding_time
        print(f"   Embeddings generate în {encoding_time:.2f} sec.")

        # 3c. Creează index FAISS
        embedding_dim = embeddings.shape[1]
        # IndexFlatIP (Inner Product) funcționează bine cu embeddings normalizate (similar cu Cosine Similarity)
        index = faiss.IndexFlatIP(embedding_dim)
        index.add(np.array(embeddings).astype('float32'))
        print(f"   Index FAISS creat (dim={embedding_dim}, {index.ntotal} vectori).")

        # 4. Rulează căutări pentru setul de referință
        if not set_referinta:
             print("   Omitere evaluare din cauza setului de referință gol.")
             continue # Treci la următorul model

        print("   Rulare căutări pe setul de referință...")
        total_search_time = 0
        num_searches = 0

        for item_ref in set_referinta:
            query_idx = item_ref["query_index"]
            # Asigură-te că indexul e valid
            if query_idx < 0 or query_idx >= len(lista_probleme_curatate):
                print(f"Avertisment: query_index {query_idx} invalid. Omitere.")
                results[model_name]["total_queries"] -= 1
                continue

            query_text = lista_probleme_curatate[query_idx] # Luăm textul din lista principală
            expected_indices = set(item_ref["expected_indices"]) # Convertim în set pentru căutare rapidă
            num_expected = len(expected_indices)

            # Generează embedding pentru query
            start_search_single = time.time()
            query_embedding = model.encode([query_text], normalize_embeddings=True)

            # Caută în index
            distances, retrieved_indices_with_dist = index.search(np.array(query_embedding).astype('float32'), K_SEARCH)
            total_search_time += (time.time() - start_search_single)
            num_searches += 1

            retrieved_indices = set(retrieved_indices_with_dist[0]) # Luăm doar indicii, convertim în set

            # Evaluare simplă: Hit@K, Precision@K, Recall@K
            # Eliminăm query-ul însuși din rezultate dacă apare (opțional, dar corect)
            retrieved_indices_no_self = retrieved_indices - {query_idx}
            # Intersecția dintre ce am găsit (fără self) și ce așteptam
            relevant_found = retrieved_indices_no_self.intersection(expected_indices)
            num_relevant_found = len(relevant_found)

            if num_relevant_found > 0:
                results[model_name]["hits_at_k"] += 1

            # Precision@K = (Relevant Găsit în Top K) / K
            precision_k = num_relevant_found / K_SEARCH if K_SEARCH > 0 else 0.0
            results[model_name]["precision_at_k_sum"] += precision_k

            # Recall@K = (Relevant Găsit în Top K) / (Total Relevant Așteptat)
            recall_k = num_relevant_found / num_expected if num_expected > 0 else 0.0
            results[model_name]["recall_at_k_sum"] += recall_k

            # Debug (poți decomenta pentru a vedea rezultatele fiecărui query)
            # print(f"      Query (idx {query_idx}): '{query_text[:50]}...'")
            # print(f"         Așteptat: {sorted(list(expected_indices))}")
            # print(f"         Găsit (top {K_SEARCH}, idx): {sorted(list(retrieved_indices_with_dist[0]))}")
            # print(f"         Relevant Găsit: {sorted(list(relevant_found))}")
            # print(f"         Hit: {'DA' if num_relevant_found > 0 else 'NU'}, Precision@{K_SEARCH}: {precision_k:.2f}, Recall@{K_SEARCH}: {recall_k:.2f}")


        # Calculează metricile medii
        if num_searches > 0:
            results[model_name]["avg_search_time"] = total_search_time / num_searches
            avg_precision = results[model_name]["precision_at_k_sum"] / num_searches
            avg_recall = results[model_name]["recall_at_k_sum"] / num_searches
            hit_rate = results[model_name]["hits_at_k"] / num_searches

            print(f"   Evaluare finalizată pentru {model_name}:")
            print(f"      Timp mediu căutare: {results[model_name]['avg_search_time']:.4f} sec")
            print(f"      Hit Rate@{K_SEARCH}:   {hit_rate:.2%}")
            print(f"      Precision@{K_SEARCH}:  {avg_precision:.4f}")
            print(f"      Recall@{K_SEARCH}:     {avg_recall:.4f}")
        else:
             print("   Nu s-au rulat căutări (set de referință gol sau invalid).")


    except Exception as e:
        print(f"EROARE în timpul procesării modelului {model_name}: {e}")
        # Poți decide să continui cu următorul model sau să oprești


# --- Pasul 5: Compară Rezultatele ---
print("\n--- Rezumat Comparativ ---")
if not results:
     print("Nu există rezultate de comparat.")
else:
    print(f"Metrici calculate pentru top {K_SEARCH} rezultate returnate.")
    print("-" * 80)
    print(f"{'Model':<25} | {'Encoding (s)':<12} | {'Avg Search (s)':<14} | {'Hit Rate':<10} | {'Precision':<10} | {'Recall':<10}")
    print("-" * 80)
    for model_name, metrics in results.items():
        enc_time = f"{metrics['encoding_time']:.2f}" if metrics['encoding_time'] is not None else "N/A"
        search_time = f"{metrics['avg_search_time']:.4f}" if metrics['avg_search_time'] is not None else "N/A"

        if metrics['total_queries'] > 0 and metrics['avg_search_time'] is not None:
            hit_rate_val = metrics['hits_at_k'] / metrics['total_queries']
            precision_val = metrics['precision_at_k_sum'] / metrics['total_queries']
            recall_val = metrics['recall_at_k_sum'] / metrics['total_queries']
            hit_rate_str = f"{hit_rate_val:.2%}"
            precision_str = f"{precision_val:.4f}"
            recall_str = f"{recall_val:.4f}"
        else:
             hit_rate_str = "N/A"
             precision_str = "N/A"
             recall_str = "N/A"


        print(f"{model_name:<25} | {enc_time:<12} | {search_time:<14} | {hit_rate_str:<10} | {precision_str:<10} | {recall_str:<10}")
    print("-" * 80)

    # Aici poți adăuga o logică simplă pentru a recomanda un model, ex:
    # best_model = max(results, key=lambda m: results[m].get('hits_at_k', -1)) # Alege pe baza hit rate-ului
    # print(f"\nModel recomandat (bazat pe Hit Rate@{K_SEARCH}): {best_model}")

1. Încărcare probleme din 'math_python_dataset_curatat.csv'...
   Am încărcat 491 probleme.

2. Definire/Încărcare set de referință...
   Set de referință definit/încărcat cu 4 intrări.

3. Testare modele de embedding (k=5)...

--- Testare Model: all-MiniLM-L6-v2 ---
   Încărcare model...


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

   Model încărcat în 6.51 sec.
   Generare embeddings pentru 491 probleme...


Batches:   0%|          | 0/16 [00:00<?, ?it/s]

   Embeddings generate în 225.48 sec.
   Index FAISS creat (dim=768, 491 vectori).
   Rulare căutări pe setul de referință...
   Evaluare finalizată pentru BAAI/bge-base-en-v1.5:
      Timp mediu căutare: 0.1212 sec
      Hit Rate@5:   0.00%
      Precision@5:  0.0000
      Recall@5:     0.0000

--- Rezumat Comparativ ---
Metrici calculate pentru top 5 rezultate returnate.
--------------------------------------------------------------------------------
Model                     | Encoding (s) | Avg Search (s) | Hit Rate   | Precision  | Recall    
--------------------------------------------------------------------------------
all-MiniLM-L6-v2          | 24.38        | 0.0220         | 0.00%      | 0.0000     | 0.0000    
BAAI/bge-base-en-v1.5     | 225.48       | 0.1212         | 0.00%      | 0.0000     | 0.0000    
--------------------------------------------------------------------------------


In [ ]:
#Pasul 4

In [1]:
!pip install langchain langchain-community sentence-transformers chromadb pandas openpyxl

In [ ]:
import pandas as pd
import time
import os
from langchain_community.document_loaders import CSVLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
# Import opțional pentru împărțire text, dacă problemele sunt lungi
# from langchain.text_splitter import RecursiveCharacterTextSplitter

# --- Configurări ---
dataset_csv_filename = "math_python_dataset_curatat.csv"

# !!! IMPORTANT: Ajustează numele coloanelor dacă sunt diferite în CSV-ul tău !!!
SOURCE_COLUMN = "problem_cleaned"     # Coloana cu textul problemei de embedat
# Asigură-te că aceste coloane chiar există în CSV!
# Poți include doar 'python_solution' dacă 'problem_latex' nu există.
METADATA_COLUMNS = ["solution_cleaned"]

EMBEDDING_MODEL_NAME = "all-MiniLM-L6-v2" # Modelul ales de voi
CHROMA_DB_PATH = "./chroma_db_math"       # Directorul unde va fi salvată baza ChromaDB
COLLECTION_NAME = "math_problems"         # Numele colecției în ChromaDB

# Determină dispozitivul (CPU sau GPU, dacă e disponibil și configurat)
# Setare manuală: DEVICE = "cpu" sau DEVICE = "cuda"
# Încearcă să detectezi automat:
try:
    import torch
    if torch.cuda.is_available():
        DEVICE = "cuda"
        print("INFO: Se va folosi GPU (cuda).")
    else:
        DEVICE = "cpu"
        print("INFO: Se va folosi CPU.")
except ImportError:
    DEVICE = "cpu"
    print("INFO: PyTorch nu este instalat, se va folosi CPU.")

print("--- Start Creare/Încărcare Vector Store ChromaDB ---")
print(f"Folosind fișierul CSV: '{dataset_csv_filename}'")
print(f"Coloana sursă pentru embedding: '{SOURCE_COLUMN}'")
print(f"Coloane metadate: {METADATA_COLUMNS}")
print(f"Model embedding: '{EMBEDDING_MODEL_NAME}' pe dispozitivul '{DEVICE}'")
print(f"Calea bazei de date ChromaDB: '{CHROMA_DB_PATH}'")
print(f"Nume colecție: '{COLLECTION_NAME}'")

# --- Pasul 1: Verifică existența CSV și a coloanelor ---
print("\n1. Verificare fișier CSV și coloane...")
if not os.path.exists(dataset_csv_filename):
    print(f"EROARE FATALĂ: Fișierul CSV '{dataset_csv_filename}' nu a fost găsit.")
    exit()

try:
    df_check = pd.read_csv(dataset_csv_filename, nrows=1) # Citeste doar header-ul
    required_cols = [SOURCE_COLUMN] + METADATA_COLUMNS
    missing_cols = [col for col in required_cols if col not in df_check.columns]
    if missing_cols:
        raise KeyError(f"Coloane lipsă în CSV: {missing_cols}")
    print("   Fișierul CSV și coloanele necesare există.")
except KeyError as e:
    print(f"EROARE FATALĂ: {e}")
    print("   Verificați numele coloanelor în script (SOURCE_COLUMN, METADATA_COLUMNS) și în fișierul CSV.")
    exit()
except Exception as e:
    print(f"EROARE FATALĂ la verificarea CSV-ului: {e}")
    exit()

# --- Pasul 2: Inițializează Funcția de Embedding ---
print(f"\n2. Inițializare model de embedding '{EMBEDDING_MODEL_NAME}'...")
start_time = time.time()
try:
    # Folosim wrapper-ul HuggingFaceEmbeddings din LangChain
    embedding_function = HuggingFaceEmbeddings(
        model_name=EMBEDDING_MODEL_NAME,
        model_kwargs={'device': DEVICE}, # Folosește CPU sau GPU
        encode_kwargs={'normalize_embeddings': True} # Normalizarea e importantă pentru similaritate cosinus
    )
    print(f"   Model de embedding inițializat. Va rula pe {DEVICE}.")
except Exception as e:
    print(f"EROARE FATALĂ la inițializarea modelului de embedding: {e}")
    print("   Verificați conexiunea la internet și dacă modelul există pe Hugging Face Hub.")
    exit()
embed_init_time = time.time() - start_time
print(f"   Timp inițializare model: {embed_init_time:.2f} secunde.")

# --- Pasul 3: Crează sau Încarcă Baza de Date Vectorială ChromaDB ---
vectorstore = None
if os.path.exists(CHROMA_DB_PATH):
    print(f"\n3. Se încearcă încărcarea bazei de date existente din '{CHROMA_DB_PATH}'...")
    try:
        vectorstore = Chroma(
            persist_directory=CHROMA_DB_PATH,
            embedding_function=embedding_function,
            collection_name=COLLECTION_NAME
        )
        print(f"   Baza de date ChromaDB încărcată cu succes. Conține {vectorstore._collection.count()} documente.")
    except Exception as e:
        print(f"   EROARE la încărcarea bazei de date existente: {e}")
        print("   Se va încerca crearea unei baze de date noi.")
        vectorstore = None # Resetează în caz de eroare la încărcare
else:
     print(f"\n3. Directorul '{CHROMA_DB_PATH}' nu există. Se va crea o bază de date nouă.")

if vectorstore is None:
    print(f"\n3. Creare bază de date nouă ChromaDB în '{CHROMA_DB_PATH}'...")
    print("   Încărcare documente din CSV...")
    start_time = time.time()
    try:
        loader = CSVLoader(
            file_path=dataset_csv_filename,
            source_column=SOURCE_COLUMN,
            metadata_columns=METADATA_COLUMNS,
            encoding='utf-8' # Asigură encoding corect
        )
        documents = loader.load()

        if not documents:
            raise ValueError("Nu s-au încărcat documente din CSV. Verificați fișierul.")

        print(f"   Am încărcat {len(documents)} documente din CSV.")

         # --- (Opțional) Împărțire Documente ---
        # Decomentează dacă textele din 'problem_cleaned' sunt lungi
        # print("   (Opțional) Împărțire documente în chunks...")
        # text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
        # docs_split = text_splitter.split_documents(documents)
        # print(f"   Documentele au fost împărțite în {len(docs_split)} chunks.")
        # data_to_index = docs_split
        # else:
        # print("   Nu se aplică împărțirea documentelor.")
        data_to_index = documents # Folosim documentele întregi

        print("   Generare embeddings și indexare în ChromaDB (acest pas poate dura)...")
        # Creează vector store-ul. Acesta va calcula embedding-urile și le va stoca.
        # Prin specificarea persist_directory, baza de date va fi salvată automat.
        vectorstore = Chroma.from_documents(
            documents=data_to_index,
            embedding=embedding_function,
            persist_directory=CHROMA_DB_PATH,
            collection_name=COLLECTION_NAME
            # Nu mai este nevoie de .persist() separat
        )
        print(f"   Baza de date ChromaDB creată și salvată cu succes. Conține {vectorstore._collection.count()} documente.")

    except Exception as e:
        print(f"EROARE FATALĂ la încărcarea documentelor sau crearea bazei ChromaDB: {e}")
        exit()
    create_db_time = time.time() - start_time
    print(f"   Timp creare bază de date: {create_db_time:.2f} secunde.")


# --- Pasul 4: Testează Căutarea ---
print("\n4. Testare Căutare Semantică...")
if vectorstore:
    test_query = "solve linear equation 3x + 5 = 11" # Modifică cu un query relevant pentru datele tale
    k_results = 3 # Numărul de rezultate de returnat
    print(f"   Query de test: '{test_query}'")
    print(f"   Se caută cele mai similare {k_results} documente...")
    start_time = time.time()
    try:
        search_results = vectorstore.similarity_search(test_query, k=k_results)
        search_time = time.time() - start_time

        if search_results:
            print(f"   {len(search_results)} rezultate găsite în {search_time:.4f} secunde:")
            for i, doc in enumerate(search_results):
                # Poți calcula și scorul dacă vectorstore-ul suportă (Chroma suportă distance)
                # score = getattr(doc, 'metadata', {}).get('_score', 'N/A') # Exemplu, poate varia
                # print(f"      Rezultat {i+1} (Score: {score}):")
                print(f"      Rezultat {i+1}:")
                print(f"         Text (din '{SOURCE_COLUMN}'): {doc.page_content[:150]}...")
                print(f"         Metadate: {doc.metadata}")
        else:
            print("      Nu s-au găsit rezultate similare pentru query-ul de test.")

    except Exception as e:
        print(f"   EROARE la efectuarea căutării semantice: {e}")
else:
    print("   EROARE: Vector store-ul nu a fost inițializat corect. Testarea căutării nu este posibilă.")

print("\n--- Proces Creare/Încărcare Vector Store ChromaDB Finalizat ---")

INFO: Se va folosi CPU.
--- Start Creare/Încărcare Vector Store ChromaDB ---
Folosind fișierul CSV: 'math_python_dataset_curatat.csv'
Coloana sursă pentru embedding: 'problem_cleaned'
Coloane metadate: ['solution_cleaned']
Model embedding: 'all-MiniLM-L6-v2' pe dispozitivul 'cpu'
Calea bazei de date ChromaDB: './chroma_db_math'
Nume colecție: 'math_problems'

1. Verificare fișier CSV și coloane...
   Fișierul CSV și coloanele necesare există.

2. Inițializare model de embedding 'all-MiniLM-L6-v2'...


<ipython-input-6-ef1b4f6b5749>:72: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function = HuggingFaceEmbeddings(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

   Model de embedding inițializat. Va rula pe cpu.
   Timp inițializare model: 36.26 secunde.

3. Directorul './chroma_db_math' nu există. Se va crea o bază de date nouă.

3. Creare bază de date nouă ChromaDB în './chroma_db_math'...
   Încărcare documente din CSV...
   Am încărcat 491 documente din CSV.
   Generare embeddings și indexare în ChromaDB (acest pas poate dura)...
   Baza de date ChromaDB creată și salvată cu succes. Conține 491 documente.
   Timp creare bază de date: 28.19 secunde.

4. Testare Căutare Semantică...
   Query de test: 'solve linear equation 3x + 5 = 11'
   Se caută cele mai similare 3 documente...
   3 rezultate găsite în 0.0299 secunde:
      Rezultat 1:
         Text (din 'problem_cleaned'): problem_cleaned: let's start with a simple math problem. what is the solution to 3x + 10 = 20...
         Metadate: {'source': "let's start with a simple math problem. what is the solution to 3x + 10 = 20", 'solution_cleaned': "<think>\nOkay, so I have this equation: 3x

In [ ]:
#Pasul 5

In [ ]:
# 1. Instalează Ollama
!curl -fsSL https://ollama.com/install.sh | sh

# 2. Pornește serverul Ollama în fundal
# Folosim nohup pentru a-l lăsa să ruleze chiar dacă celula se termină
# Salvăm logurile în ollama.log
!nohup ollama serve > ollama.log 2>&1 &

# 3. Așteaptă puțin pentru ca serverul să pornească complet (esențial!)
import time
print("Așteptare 5 secunde pentru pornirea serverului Ollama...")
time.sleep(5)
print("Serverul Ollama ar trebui să fie pornit.")

# 4. (Opțional) Verifică dacă serverul rulează (ar trebui să afișeze o listă goală sau modele dacă ai descărcat deja)
!ollama list

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
Așteptare 5 secunde pentru pornirea serverului Ollama...
Serverul Ollama ar trebui să fie pornit.
NAME    ID    SIZE    MODIFIED 


In [ ]:
# Descarcă modelul DeepSeek ales (ex: deepseek-coder:6.7b)
# Acest pas poate dura câteva minute în funcție de mărimea modelului și viteza Colab
print("Descărcare model deepseek-coder:6.7b (poate dura)...")
!ollama pull deepseek-coder:6.7b
print("Model descărcat.")

# Verifică din nou lista să vezi dacă modelul apare
!ollama list

Descărcare model deepseek-coder:6.7b (poate dura)...
pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest 
pulling 59bb50d8116b...   0% ▕▏    0 B/3.8 GB                  pulling manifest 
pulling 59bb50d8116b...   0% ▕▏  90 KB/3.8 GB                  pulling manifest 
pulling 59bb50d8116b...   0% ▕▏ 6.7 MB/3.8 GB                  pulling manifest 
pulling 59bb50d8116b...   1% ▕▏  34 MB/3.8 GB                  pulling manifest 
pulling 59bb50d8116b...   2% ▕▏  67 MB/3.8 GB                  pulling manifest 
pulling 59bb50d8116b...   3% ▕▏ 113 MB/3.8 GB                  pulling manifest 
pulling 59bb50d8116b...   3% ▕▏ 125 MB/3.8 GB                  pulling manifest 
pulling 59bb50d8116b...   4% ▕▏ 155 MB/3.8 GB                  pulling manifest 
pulling 59bb50d8116b...   4% ▕▏ 156 MB/3.8 GB                  pulling manifest 
pulling 59bb50d8116b...   4% ▕▏ 157 MB/3.8 GB                  pulling manifest 
pulli

In [ ]:
!ollama pull mistral:7b
print("Model descărcat.")

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest 
pulling ff82381e2bea...   0% ▕▏    0 B/4.1 GB                  pulling manifest 
pulling ff82381e2bea...   0% ▕▏    0 B/4.1 GB                  pulling manifest 
pulling ff82381e2bea...   0% ▕▏ 1.6 MB/4.1 GB                  pulling manifest 
pulling ff82381e2bea...   1% ▕▏  39 MB/4.1 GB                  pulling manifest 
pulling ff82381e2bea...   2% ▕▏  68 MB/4.1 GB                  pulling manifest 
pulling ff82381e2bea...   3% ▕▏ 108 MB/4.1 GB                  pulling manifest 
pulling ff82381e2bea...   3% ▕▏ 134 MB/4.1 GB                  pulling manifest 
pulling ff82381e2bea...   4% ▕▏ 151 MB/4.1 GB                  pulling manifest 
pulling ff82381e2bea...   4% ▕▏ 166 MB/4.1 GB                  pulling manifest 
pulling ff82381e2bea...   5% ▕▏ 188 MB/4.1 GB                  pulling manifest 
pulling ff82381e2bea...   5% ▕▏ 202 MB/4.1 GB                  pulling manife

In [ ]:
!ollama rm deepseek-coder:6.7b

# Șterge modelul mistral cu tag-ul 7b (dacă l-ai descărcat explicit)
!ollama rm mistral:7b


deleted 'deepseek-coder:6.7b'
deleted 'mistral:7b'


In [ ]:
!ollama pull phi
print("Model descărcat.")

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest 
pulling 04778965089b...   0% ▕▏    0 B/1.6 GB                  pulling manifest 
pulling 04778965089b...   0% ▕▏    0 B/1.6 GB                  pulling manifest 
pulling 04778965089b...   0% ▕▏ 4.0 MB/1.6 GB                  pulling manifest 
pulling 04778965089b...   3% ▕▏  44 MB/1.6 GB                  pulling manifest 
pulling 04778965089b...   5% ▕▏  74 MB/1.6 GB                  pulling manifest 
pulling 04778965089b...   7% ▕▏ 119 MB/1.6 GB                  pulling manifest 
pulling 04778965089b...  10% ▕▏ 153 MB/1.6 GB                  pulling manifest 
pulling 04778965089b...  11% ▕▏ 169 MB/1.6 GB                  pulling manifest 
pulling 04778965089b...  12% ▕▏ 200 MB/1.6 GB                  pulling manifest 
pulling 04778965089b...  14% ▕▏ 220 MB/1.6 GB                  pulling manifest 
pulling 04778965089b...  15% ▕▏ 237 MB/1.6 GB                  pulling manifest 
pulling 0477896

In [ ]:
!ollama list

NAME          ID              SIZE      MODIFIED      
phi:latest    e2fd6321a5fe    1.6 GB    2 seconds ago    


In [ ]:
!pip install langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.6/433.6 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 25.9 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.51
    Uninstalling langchain-core-0.3.51:
      Successfully uninstalled langchain-core-0.3.51


In [2]:
# ==============================================================================
# PASUL 0: Instalare Biblioteci Necesare pentru Cloud API
# ==============================================================================
# ==============================================================================
# PASUL 1: Importuri și Configurări
# ==============================================================================
import pandas as pd
import time
import os
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser


# Varianta 1: OpenAI
from langchain_openai import ChatOpenAI


# --- Configurări Principale ---
CSV_FILE_PATH = "math_python_dataset_curatat.csv"
CHROMA_DB_PATH = "./chroma_db_math"
COLLECTION_NAME = "math_problems"
EMBEDDING_MODEL_NAME = "all-MiniLM-L6-v2"
# NUM_TEST_QUESTIONS = 10 # Comentat - acum testăm doar o întrebare
TEST_QUESTION_INDEX = 5  # Indexul întrebării de testat

# --- Determină Dispozitiv pentru Embeddings ---
try:
    import torch
    if torch.cuda.is_available():
        DEVICE = "cuda"
        print(f"INFO: Se va folosi GPU ({DEVICE}) pentru embeddings.")
    else:
        DEVICE = "cpu"
        print(f"INFO: Se va folosi CPU ({DEVICE}) pentru embeddings.")
except ImportError:
    DEVICE = "cpu"
    print("INFO: PyTorch nu este instalat sau nu este necesar, se va folosi CPU pentru embeddings.")

# ==============================================================================
# PASUL 2: Inițializare Componente RAG
# ==============================================================================

# --- Inițializare Funcție Embedding ---
print(f"\n1. Inițializare model embedding '{EMBEDDING_MODEL_NAME}'...")
try:
    embedding_function = HuggingFaceEmbeddings(
        model_name=EMBEDDING_MODEL_NAME,
        model_kwargs={'device': DEVICE},
        encode_kwargs={'normalize_embeddings': True}
    )
    print(f"   Model embedding inițializat pe {DEVICE}.")
except Exception as e:
    print(f"   EROARE FATALĂ la inițializarea modelului de embedding: {e}")
    exit()

# --- Încărcare Vector Store ChromaDB ---
# (Rămâne la fel)
print(f"\n2. Încărcare ChromaDB din '{CHROMA_DB_PATH}'...")
if not os.path.exists(CHROMA_DB_PATH):
    print(f"   EROARE FATALĂ: Directorul ChromaDB '{CHROMA_DB_PATH}' nu a fost găsit.")
    exit()
try:
    vectorstore = Chroma(
        persist_directory=CHROMA_DB_PATH,
        embedding_function=embedding_function,
        collection_name=COLLECTION_NAME
    )
    print(f"   Baza de date ChromaDB încărcată. Conține {vectorstore._collection.count()} documente.")
except Exception as e:
    print(f"   EROARE FATALĂ la încărcarea ChromaDB: {e}")
    exit()

# --- Creare Retriever ---
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}
)
print("   Retriever creat (k=3).")


# --- Inițializare LLM (Folosind API Cloud) ---
print(f"\n3. Inițializare LLM folosind API Cloud...")

llm = None # Inițializăm cu None

# --- Varianta 1: OpenAI ---
# Necesită cheia API salvată în Colab Secrets ca 'OPENAI_API_KEY'
try:
    from google.colab import userdata
    openai_api_key = userdata.get('OPENAI_API_KEY')
    if not openai_api_key:
        raise ValueError("Cheia API OpenAI ('OPENAI_API_KEY') nu a fost găsită în secretele Colab.")
    os.environ["OPENAI_API_KEY"] = openai_api_key # Setează variabila de mediu

    # Alege modelul OpenAI dorit (gpt-3.5-turbo e o opțiune bună și ieftină)
    OPENAI_MODEL_NAME = "gpt-3.5-turbo"
    llm = ChatOpenAI(
        model_name=OPENAI_MODEL_NAME,
        temperature=0.1 # Temperatură mică pentru cod
    )
    print(f"   LLM OpenAI ({OPENAI_MODEL_NAME}) inițializat cu succes.")

except ImportError:
     print("   AVERTISMENT: Modulul 'google.colab.userdata' nu poate fi importat.")
     print("                Setează manual variabila de mediu 'OPENAI_API_KEY' dacă nu ești în Colab.")
     # Ar trebui să verifici dacă os.environ["OPENAI_API_KEY"] există dacă nu ești în Colab
     if "OPENAI_API_KEY" not in os.environ:
          print("   EROARE: Variabila de mediu 'OPENAI_API_KEY' nu este setată.")
     else:
           OPENAI_MODEL_NAME = "gpt-3.5-turbo"
           llm = ChatOpenAI(model_name=OPENAI_MODEL_NAME, temperature=0.1)
           print(f"   LLM OpenAI ({OPENAI_MODEL_NAME}) inițializat (folosind variabila de mediu).")

except ValueError as ve:
    print(f"   EROARE la configurarea OpenAI: {ve}")
except Exception as e:
    print(f"   EROARE FATALĂ la inițializarea LLM OpenAI: {e}")



# --- Verificare Finală LLM ---
if llm is None:
    print("\n   !!! EROARE FATALĂ: Niciun LLM Cloud nu a fost inițializat cu succes.")
    print("       Verifică dacă ai decomentat una dintre variante (OpenAI sau Gemini) și ai configurat corect cheia API în secretele Colab.")
    exit()


# --- Definire Prompt Template ---
# (Rămâne la fel)
prompt_template = """
You are an expert Python programmer specialized in mathematical libraries like NumPy and SymPy.
Use the provided context (examples of similar math problems and their Python solutions) to generate Python code that solves or represents the given mathematical problem described in natural language.

Context:
---------------------
{context}
---------------------

Mathematical Problem / Question:
{question}

Instructions:
1. Analyze the problem description.
2. Use relevant examples from the context if helpful.
3. Generate concise, runnable Python code using NumPy or SymPy.
4. Add minimal comments if necessary.
5. If you cannot solve it, state that clearly.

Python Code:
"""
prompt = PromptTemplate(template=prompt_template,
                        input_variables=["context", "question"])
print("\n4. Prompt template definit.")

# --- Funcție Formatare Documente pentru Context ---
# (Rămâne la fel - verifică doar numele metadatei 'solution_cleaned')
def format_docs(docs):
    """Formatează documentele returnate de retriever pentru promptul LLM."""
    formatted_list = []
    for i, doc in enumerate(docs):
        content = doc.page_content
        solution = doc.metadata.get('solution_cleaned', 'N/A: Metadata "solution_cleaned" not found')
        latex = doc.metadata.get('problem_latex', 'N/A')
        formatted_list.append(
            f"Example {i+1}:\n"
            f"Problem Description: {content}\n"
            # f"LaTeX Version (if available): {latex}\n"
            f"Solution:\n```python\n{solution}\n```"
        )
    return "\n\n".join(formatted_list) if formatted_list else "No relevant examples found in the context."

# --- Construire Lanț RAG cu LCEL ---
# (Rămâne la fel, folosește variabila `llm` actualizată)
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
print("\n5. Lanțul RAG a fost construit (folosind LLM Cloud).")


# ==============================================================================
# ==============================================================================
# PASUL 3: Testare Folosind o Întrebare Introdusă de Utilizator
# ==============================================================================

print(f"\n6. Așteptare întrebare de la utilizator...")

# --- Obține Întrebarea de la Consolă ---
try:
    single_test_question = input(">>> Introduceți problema matematică (în limbaj natural) și apăsați Enter: ")
    # Verifică dacă utilizatorul a introdus ceva
    if not single_test_question or not single_test_question.strip():
        print("\n   EROARE: Nu a fost introdusă nicio întrebare validă. Se oprește.")
        exit()
    single_test_question = single_test_question.strip() # Elimină spațiile goale de la început/sfârșit
    print(f"   Întrebare primită: '{single_test_question}'")

except EOFError:
    # Aceasta poate apărea în unele medii dacă input-ul este redirecționat
    print("\n   EROARE: Eroare la citirea input-ului (EOFError). Se oprește.")
    exit()
except KeyboardInterrupt:
    # Permite oprirea curată cu Ctrl+C
    print("\n   Proces întrerupt de utilizator. La revedere!")
    exit()


# --- Rulare Test Unic și Stocare Rezultat ---
print(f"\n7. Rulare test pentru întrebarea introdusă...")
single_result = {} # Dicționar pentru a stoca rezultatul unic

start_testing_time = time.time() # Timpul de start pentru singurul test

generated_code = None
error_message = None
invoke_time = 0.0


print(f"\n--- Test Unic ---")
print(f"Întrebare: {single_test_question}")

start_invoke_time = time.time()

try:

    print("\nGenerare Cod Python (folosind API Cloud)...")
    generated_code = rag_chain.invoke(single_test_question)
    invoke_time = time.time() - start_invoke_time
    print(f"Răspuns generat în {invoke_time:.2f} secunde.")

except Exception as e:
    invoke_time = time.time() - start_invoke_time
    error_message = str(e)
    print(f"!!! EROARE la generare (API Cloud) în {invoke_time:.2f} secunde:")
    print(f"    {error_message}")



single_result = {
    "question": single_test_question,
    "generated_code": generated_code.strip() if generated_code else None,
    "error": error_message,
    "time_taken_invoke": invoke_time
}
print("-" * 40)


end_testing_time = time.time()
total_testing_time = end_testing_time - start_testing_time

print("\n--- Procesare Întrebare Finalizată ---")
print(f"Timp total pentru procesare și generare: {total_testing_time:.2f} secunde.")

# ==============================================================================
# PASUL 4: Afișare Rezultat Unic Detaliat
# ==============================================================================
print("\n--- Rezumatul Rezultatului ---")

if single_result:
    print(f"  -> Întrebare Utilizator: {single_result['question']}")
    if single_result['error']:
        print(f"  -> EROARE la Generare:   {single_result['error']}")
    elif single_result['generated_code']:
         print(f"  -> Cod Generat:\n```python\n{single_result['generated_code']}\n```")
    else:
        print("  -> Cod Generat:         Nu a fost returnat niciun cod și nicio eroare explicită.")
    print(f"  -> Timp Generare LLM:   {single_result['time_taken_invoke']:.2f}s")
else:
    print("Nu s-a putut procesa întrebarea (verificați erorile anterioare).")

print("\n--- Script Finalizat ---")

ModuleNotFoundError: No module named 'langchain_openai'

In [ ]:
!pip install --upgrade langchain langchain-core langchain-community langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.6/433.6 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 27.6 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.51
    Uninstalling langchain-core-0.3.51:
      Successfully uninstalled langchain-core-0.3.51
